In [2]:
#create a directory named '/.kaggle/
!mkdir ~/.kaggle/

In [1]:
from google.colab import files

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install kaggle

In [4]:

!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d thoughtvector/customer-support-on-twitter

 99% 167M/169M [00:06<00:00, 33.3MB/s]
100% 169M/169M [00:06<00:00, 27.4MB/s]


In [6]:
!unzip /content/customer-support-on-twitter.zip

Archive:  /content/customer-support-on-twitter.zip
  inflating: sample.csv              
  inflating: twcs/twcs.csv           


In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df=pd.read_csv('/content/sample.csv')

In [45]:
#quest1 transferer en miniscule
df['text_cleaned'] = df['text'].apply(lambda x: str(x).lower())
print(df.head(30))

    tweet_id        author_id  inbound                      created_at  \
0     119237           105834     True  Wed Oct 11 06:55:44 +0000 2017   
1     119238     ChaseSupport    False  Wed Oct 11 13:25:49 +0000 2017   
2     119239           105835     True  Wed Oct 11 13:00:09 +0000 2017   
3     119240     VirginTrains    False  Tue Oct 10 15:16:08 +0000 2017   
4     119241           105836     True  Tue Oct 10 15:17:21 +0000 2017   
5     119243     VirginTrains    False  Tue Oct 10 15:25:14 +0000 2017   
6     119244           105836     True  Tue Oct 10 15:26:44 +0000 2017   
7     119245     VirginTrains    False  Tue Oct 10 15:33:22 +0000 2017   
8     119242           105836     True  Tue Oct 10 15:09:00 +0000 2017   
9     119246     VirginTrains    False  Tue Oct 10 10:13:19 +0000 2017   
10    119248     AppleSupport    False  Wed Oct 11 13:38:29 +0000 2017   
11    119249           105837     True  Wed Oct 11 07:37:27 +0000 2017   
12    119250           105838     True

In [31]:
#supprimer les ponctuation
import string
l=df['text']
df['text_cleaned']=l.apply(lambda x:x.translate(str.maketrans('','',string.punctuation)))

print(df['text_cleaned'])

0     AppleSupport causing the reply to be disregard...
1     105835 Your business means a lot to us Please ...
2     76328 I really hope you all change but Im sure...
3     105836 LiveChat is online at the moment  https...
4     VirginTrains see attached error message Ive tr...
                            ...                        
88    105860 I wish Amazon had an option of where I ...
89    They reschedule my shit for tomorrow httpstcoR...
90    105861 Hey Sara sorry to hear of the issues yo...
91    Tesco bit of both  finding the layout cumberso...
92    105861 If that doesnt help please DM your full...
Name: text_cleaned, Length: 93, dtype: object


In [50]:
import nltk
from nltk.corpus import stopwords

# Téléchargement des mots vides
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
l=df['text']
# Filtrage des mots vides du texte
df['text_cleaned'] =l.apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
from collections import Counter

# Identifier les mots fréquents
word_freq = Counter(' '.join(df['text_cleaned']).split())
most_common_words = [word for word, freq in word_freq.most_common(10)] # Exemple : les 10 mots les plus courants

# Supprimer les mots fréquents
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in most_common_words]))
frequ=df['text_cleaned']
print(frequ.head(30))

0     causing reply disregarded tapped notification ...
1     @105835 Your business means lot us. Please nam...
2     @76328 really hope change I'm sure won't! Beca...
3     @105836 LiveChat online moment https://t.co/SY...
4     @VirginTrains see attached error message. I've...
5         @105836 Have tried another device, Miriam ^MM
6     @VirginTrains yep, I've tried laptop several t...
7     @105836 It's working OK here, Miriam. Does lin...
8     @VirginTrains still heard &amp; number I'm dir...
9     @105836 That's we're Miriam 😊 The team send em...
10    @105837 We help. Which version iOS on? You fin...
11            @105838 Me suffering , hope find solution
12    hi #apple, I’ve concern latest ios slow #iphon...
13    @105839 reaching us. We always happy help. Sen...
14    updated suddenly everything takes ages load wt...
15    @105840 there! What device happening on? If co...
16    @SpotifyCares Thanks! Version 8.4.22.857 armv7...
17    @105840 Thanks. The distance could possibl

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [51]:
# Identifier les mots rares
rare_words = [word for word, freq in word_freq.items() if freq == 1] # Exemple : les mots qui n'apparaissent qu'une fois

# Supprimer les mots rares
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([word for word in x.split() if word not in rare_words]))
rare=df['text_cleaned']
print(rare.head(30))

0                                                 reply
1                       means lot us. Please name, code
2                                              hope I'm
3             @105836 moment 031 031 option 3 call back
4     @VirginTrains see I've tried several times pas...
5                      @105836 tried device, Miriam ^MM
6     @VirginTrains I've tried several times past we...
7                    @105836 It's working Does link ^MM
8     @VirginTrains still &amp; number I'm &amp; Can...
9          @105836 That's we're Miriam 😊 The send email
10    We help. Which version iOS on? find &gt; &gt; DM.
11                                   hope find solution
12                    latest happy it. solution please?
13    reaching us. We always happy help. Send look t...
14            updated everything load @76099 update fix
15    @105840 What device happening on? If could let...
16    @SpotifyCares Thanks! bluetooth speaker Does d...
17    @105840 The distance could playback. Does 

In [52]:
# Utilisation d'expressions régulières pour supprimer ces éléments du texte
import re
# Supprimer les balises HTML
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'<.*?>', '', x))
print(df['text_cleaned'])

0                                                 reply
1                       means lot us. Please name, code
2                                              hope I'm
3             @105836 moment 031 031 option 3 call back
4     @VirginTrains see I've tried several times pas...
                            ...                        
88                                 option get store lot
89                                                 They
90                        @105861 Hey sorry hear issues
91                                   @Tesco getting Not
92    @105861 If please full name, address email dev...
Name: text_cleaned, Length: 93, dtype: object


In [53]:
# Supprimer les URL
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'http\S+|www\S+', '', x))
print(df['text_cleaned'])

0                                                 reply
1                       means lot us. Please name, code
2                                              hope I'm
3             @105836 moment 031 031 option 3 call back
4     @VirginTrains see I've tried several times pas...
                            ...                        
88                                 option get store lot
89                                                 They
90                        @105861 Hey sorry hear issues
91                                   @Tesco getting Not
92    @105861 If please full name, address email dev...
Name: text_cleaned, Length: 93, dtype: object


In [54]:
# Supprimer les émojis
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: re.sub(r'[\U00010000-\U0010ffff]', '', x))
print(df['text_cleaned'])

0                                                 reply
1                       means lot us. Please name, code
2                                              hope I'm
3             @105836 moment 031 031 option 3 call back
4     @VirginTrains see I've tried several times pas...
                            ...                        
88                                 option get store lot
89                                                 They
90                        @105861 Hey sorry hear issues
91                                   @Tesco getting Not
92    @105861 If please full name, address email dev...
Name: text_cleaned, Length: 93, dtype: object


In [55]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Téléchargement des ressources nécessaires pour la lemmatisation
nltk.download('punkt')
nltk.download('wordnet')

# Stemming avec NLTK
stemmer = PorterStemmer()
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([stemmer.stem(word) for word in word_tokenize(x)]))
print(df['text_cleaned'])


0                                                 repli
1                       mean lot us . pleas name , code
2                                             hope i 'm
3            @ 105836 moment 031 031 option 3 call back
4      @ virgintrain see i 've tri sever time past week
                            ...                        
88                                 option get store lot
89                                                 they
90                         @ 105861 hey sorri hear issu
91                                      @ tesco get not
92    @ 105861 if pleas full name , address email de...
Name: text_cleaned, Length: 93, dtype: object


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [56]:
# Lemmatisation avec NLTK
lemmatizer = WordNetLemmatizer()
df['text_cleaned'] = df['text_cleaned'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(x)]))
print(df['text_cleaned'])

0                                                 repli
1                         mean lot u . plea name , code
2                                             hope i 'm
3            @ 105836 moment 031 031 option 3 call back
4      @ virgintrain see i 've tri sever time past week
                            ...                        
88                                 option get store lot
89                                                 they
90                         @ 105861 hey sorri hear issu
91                                      @ tesco get not
92    @ 105861 if plea full name , address email dev...
Name: text_cleaned, Length: 93, dtype: object


In [57]:

#resultat
import pandas as pd
text_nv = df['text_cleaned']
# Charger le DataFrame d'origine
df = pd.read_csv('/content/sample.csv')

# Créer un nouveau DataFrame avec les colonnes tweet_id, text et text_cleaned
df_processed = pd.DataFrame({
    'tweet_id': df['tweet_id'],  # Colonne tweet_id du DataFrame d'origine
    'text': df['text'],           # Colonne text du DataFrame d'origine
    'text_cleaned': text_nv  # Nouvelle colonne text_cleaned après prétraitement
})

#afficher le nouveau dataframe
df_processed

,tweet_id,text,text_cleaned
0,119237,@AppleSupport causing the reply to be disregar...,repli
1,119238,@105835 Your business means a lot to us. Pleas...,"mean lot u . plea name , code"
2,119239,@76328 I really hope you all change but I'm su...,hope i 'm
3,119240,@105836 LiveChat is online at the moment - htt...,@ 105836 moment 031 031 option 3 call back
4,119241,@VirginTrains see attached error message. I've...,@ virgintrain see i 've tri sever time past week
...,...,...,...
88,119330,@105860 I wish Amazon had an option of where I...,option get store lot
89,119331,They reschedule my shit for tomorrow https://t...,they
90,119332,"@105861 Hey Sara, sorry to hear of the issues ...",@ 105861 hey sorri hear issu
91,119333,@Tesco bit of both - finding the layout cumber...,@ tesco get not
